In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("2017_2019.csv")
df.drop(['Unnamed: 18'],inplace=True,axis=1)

In [3]:
dates = set()
for i,data in df.iterrows():
    dates.add(pd.to_datetime( 
                             str(data['Year'])[:-2]+'-'+str(data['Month'])[:-2] + '-' + str(data['Day'])[:-2],
                             format="%Y-%m-%d"
                             ))

In [4]:
def get_day(df:pd.DataFrame,day:pd.Timestamp) -> pd.DataFrame:
    
    return df.loc[ (df ['Year'] == day.year) & 
                  (df['Day'] == day.day) &
                  (df['Month'] == day.month)]
    
def get_records_night(df:pd.DataFrame,day:pd.Timestamp):
    records_beafore_df = get_n_days_beafore(df,day,1)
    records_beafore_df = records_beafore_df.loc[(records_beafore_df.Hour > 12)  & (records_beafore_df["Clearsky DNI"] != 0)]
    
    records_df = get_day(df,day)
    records_df = records_df.loc[(records_df.Hour < 12) &( records_df.DNI == 0) &(records_df["Clearsky DNI"] != 0)]
    
    return(pd.concat([records_beafore_df,records_df]))
    
    
  
def get_n_days_beafore(df:pd.DataFrame,day:pd.Timestamp,n:int):
    day_beafore = day - pd.offsets.Day(n)
    return get_day(df,day_beafore)

def get_working_day(df:pd.DataFrame,day:pd.Timestamp):
    records = get_day(df,day)
    return records.loc[(records.DNI != 0) | (records["Clearsky DNI"] != 0)]

    
    

In [5]:
_ = pd.Series( list(dates)).sort_values()
get_records_night(df,_.iloc[2])

,Year,Month,Day,Hour,Minute,Temperature,Clearsky DHI,Clearsky DNI,Clearsky GHI,Dew Point,DHI,DNI,GHI,Relative Humidity,Solar Zenith Angle,Surface Albedo,Pressure,Wind Speed
148,2017,1,2,13,0,7.5,69,738,296,-1.7,69,738,296,52.30,72.09,0.13,1000,0.9
149,2017,1,2,13,15,7.2,65,702,258,-1.7,65,702,258,53.38,74.03,0.13,1000,0.8
150,2017,1,2,13,30,6.8,60,658,219,-1.7,60,658,219,54.86,76.07,0.13,1000,0.8
151,2017,1,2,13,45,6.5,54,610,179,-2.4,54,610,179,52.89,78.21,0.13,1000,0.8
152,2017,1,2,14,0,6.1,48,542,138,-2.4,48,542,138,54.36,80.44,0.13,1000,0.7
153,2017,1,2,14,15,6.0,40,455,98,-2.4,40,455,98,54.74,82.74,0.13,1000,0.7
154,2017,1,2,14,30,5.9,31,342,60,-2.4,31,342,60,55.12,85.11,0.13,1000,0.6
155,2017,1,2,14,45,5.7,19,199,28,-3.1,19,199,28,52.99,87.49,0.13,1000,0.6
215,2017,1,3,5,45,3.8,19,155,25,0.7,9,0,9,80.07,87.59,0.13,1000,3.0
216,2017,1,3,6,0,4.3,31,285,55,0.7,20,0,20,77.31,85.21,0.13,1000,3.1


In [6]:
df[df.isna().any(axis=1)]

,Year,Month,Day,Hour,Minute,Temperature,Clearsky DHI,Clearsky DNI,Clearsky GHI,Dew Point,DHI,DNI,GHI,Relative Humidity,Solar Zenith Angle,Surface Albedo,Pressure,Wind Speed


In [88]:
records_df =pd.DataFrame()
for idx,date in enumerate( pd.Series( list(dates)).sort_values().values):
    
    date = pd.to_datetime(date)
    day = get_working_day(df,date)
    record = pd.DataFrame([{'date':date}])
    
    record['month'] = date.month
    record['len_day'] = len(day)
    record['temp_mean'] = day.Temperature.mean()
    record['press_mean'] = day.Pressure.mean()
    record['wind_mean'] = day["Wind Speed"].mean()
    
    
    night = get_records_night(df,date)
    
    if idx == 0:
        record['len_night'] = 0
    else:
        record['len_night'] = len(night)
        record['night_temp_mean'] = night.Temperature.mean()
        record['night_press_mean'] = night.Pressure.mean()
        record['night_wind_mean'] = night["Wind Speed"].mean()
    
    record['Y'] = (day["Clearsky DNI"].sum() + day['DNI'].sum()) / record['len_day']
    records_df = pd.concat([records_df,record])
records_df

,date,month,len_day,temp_mean,press_mean,wind_mean,len_night,Y,night_temp_mean,night_press_mean,night_wind_mean
0,2017-01-01,1,37,4.910811,1001.648649,1.878378,0,1397.243243,NaN,NaN,NaN
0,2017-01-02,1,37,6.035135,1000.918919,1.335135,11,1391.621622,3.863636,1000.727273,1.709091
0,2017-01-03,1,37,8.983784,999.756757,2.970270,14,1068.054054,5.828571,1000.285714,1.835714
0,2017-01-04,1,37,10.167568,996.621622,3.432432,15,685.891892,8.826667,998.400000,2.533333
0,2017-01-05,1,37,14.021622,988.594595,6.070270,26,538.972973,12.834615,990.846154,5.188462
...,...,...,...,...,...,...,...,...,...,...,...
0,2019-12-27,12,37,8.959459,996.324324,3.143243,8,1206.540541,9.762500,989.875000,3.962500
0,2019-12-28,12,37,8.727027,991.810811,1.532432,22,595.216216,8.468182,993.545455,1.740909
0,2019-12-29,12,37,7.818919,992.486486,4.994595,19,636.756757,6.684211,992.157895,3.405263
0,2019-12-30,12,37,7.432432,987.621622,6.756757,33,566.675676,7.439394,988.303030,6.748485


In [20]:
records_df[records_df.isna().any(axis=1)]

,date,month,len_day,temp_mean,press_mean,wind_mean,len_night,Y,night_temp_mean,night_press_mean,night_wind_mean
0,2017-01-01,1,37,4.910811,1001.648649,1.878378,0,1397.243243,NaN,NaN,NaN


In [111]:
def get_Data_n_records(df,num):
    X = []
    Y = []
    for i in range(len(df)):
        if i < num+1:
            pass
        else:
            X.append(df.iloc[i-num-1:i-1].values)
            Y.append(df.iloc[i].Y)
    
    
    return np.array(X),np.array(Y)

X,Y = get_Data_n_records(records_df,3)

In [52]:
import pandas as pd
import numpy as np 
import keras as kr
import sklearn as skl
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import tensorflow as tf
from sklearn.preprocessing import normalize
from sklearn.metrics import recall_score

In [91]:
records_df.drop(["date"], inplace= True,axis= 1)
records_df.dropna(inplace=True)

In [92]:
records_df[records_df.isna().any(axis=1)]

,month,len_day,temp_mean,press_mean,wind_mean,len_night,Y,night_temp_mean,night_press_mean,night_wind_mean


In [120]:
num = 30
X,Y = get_Data_n_records(records_df,num)
scaler = skl.preprocessing.MinMaxScaler(feature_range=(0, 1))
scaler.fit(records_df)


Y = pd.Series(Y)
Y=(Y-Y.min())/(Y.max()-Y.min())
x_train,x_test,y_train,y_test = train_test_split(X,Y)
x_train,x_val,y_train,y_val = train_test_split(x_train,y_train)

# regr = RandomForestRegressor(random_state=0)
# regr.fit(x_train, y_train)

model = kr.Sequential()
model.add(kr.layers.Dense(num,input_shape=(num, 10)))
model.add(kr.layers.LSTM(10))
model.add(kr.layers.Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam',metrics=tf.keras.metrics.MeanAbsoluteError())
hist = model.fit(x_train, y_train, epochs=200, batch_size=16, validation_data=(x_val, y_val), 
        callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=25, restore_best_weights=False)])

Epoch 1/200
38/38 [==============================] - 3s 18ms/step - loss: 0.1724 - mean_absolute_error: 0.3160 - val_loss: 0.1115 - val_mean_absolute_error: 0.2222
Epoch 2/200
38/38 [==============================] - 0s 9ms/step - loss: 0.0977 - mean_absolute_error: 0.2220 - val_loss: 0.0648 - val_mean_absolute_error: 0.1841
Epoch 3/200
38/38 [==============================] - 0s 9ms/step - loss: 0.0605 - mean_absolute_error: 0.1966 - val_loss: 0.0488 - val_mean_absolute_error: 0.1739
Epoch 4/200
38/38 [==============================] - 0s 9ms/step - loss: 0.0488 - mean_absolute_error: 0.1800 - val_loss: 0.0485 - val_mean_absolute_error: 0.1789
Epoch 5/200
38/38 [==============================] - 0s 9ms/step - loss: 0.0472 - mean_absolute_error: 0.1838 - val_loss: 0.0459 - val_mean_absolute_error: 0.1787
Epoch 6/200
38/38 [==============================] - 0s 9ms/step - loss: 0.0466 - mean_absolute_error: 0.1851 - val_loss: 0.0485 - val_mean_absolute_error: 0.1868
Epoch 7/200
38/38 [==

In [110]:
#recall_score(y_test.values,mlp_model.predict(x_test).reshape(1,-1))

mlp_model.predict(x_test).reshape(1,-1),y_test

9/9 [==============================] - 0s 806us/step


(array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.